### Stars of the Hipparcos Catalogue
#### Writing an efficient Python code for large data sets

Astronomers and astrophysicists work with many different data sets obtained through ground-based and space-based telescopes. The volume of these data sets can be so large that includes over a million gigabytes of information. 

In 1989, the European Space Agency launched the Hipparcos satellite (HIgh Precision PARallax COllecting Satellite) to measure the parallax and proper motions of stars. The satellite was able to measure parallaxes to an order of magnitude better precision than previous methods. 

By providing positions, distances and proper motions for 118,218 stars to an average accuracy of 1 milliarcsecond, the satellite helped us to understand and calibrate the distance scale of the universe.

The hip_sp.csv is a short version of the main Hipparcos catalog. The hip_sp.csv file contains 118218 rows and 11 columns, selected out of 78 columns from the main Hipparcos catalog. The selected columns are:

<ul>    
<li> Hip_No -- unique Hipparcos number </li>
<li> Alpha in (h,m,s) & Delta in (d,m,s)-- right ascension and declination represent the stellar coordinates </li>
<li> Vmag -- visual magnitude is a measure of the apparent stellar brightness </li>
<li> B-V and V-I -- color indexes indicate star's color </li>
<li> Plx -- trigonometric parallax in milliarcseconds is a small apparent angular shift in the position of a star on the sky due to Earth's motion around the Sun </li>
<li> e_Plx -- standard error in Plx in milliarcseconds </li>
<li> Var_period -- a period (in days) for variable stars </li>
<li> Var_type -- type of variability </li>
<li> Spectral_type -- a spectral type of an object.  This quantity is a measure of the stellar temperature and color. </li>
</ul>

### Importing data 

With errors = 'coerce'  invalid parsing is set to 'NaN'

In [1]:
#importing Python libraries
import numpy as np
import pandas as pd

#defining path
file = '../data/hip_sp.csv'

#defining column names
new_column_names = ['Hip_No', 'Alpha', 'Delta','Vmag', 'Plx', 'e_Plx', 'B-V', 'V-I', 'Var_period', 'Var_type','Spectral_type']

#importing data
hip_all_stars = pd.read_csv(file, header = None, sep =',',
                usecols = [1,2,3,4,5,6,7,8,9,10,11],  
                names = new_column_names,
                low_memory = False)

#changing column types
col_list = ['Vmag', 'Plx', 'e_Plx', 'B-V', 'V-I']
for  col in col_list:
   hip_all_stars[col] = pd.to_numeric(hip_all_stars[col],  errors = 'coerce')

#displaying data frame
hip_all_stars.head(5)

,Hip_No,Alpha,Delta,Vmag,Plx,e_Plx,B-V,V-I,Var_period,Var_type,Spectral_type
0,1,00 00 00.22,+01 05 20.4,9.10,3.54,1.39,0.482,0.55,,,F5
1,2,00 00 00.91,-19 29 55.8,9.27,21.90,3.10,0.999,1.04,,C,K3V
2,3,00 00 01.20,+38 51 33.4,6.61,2.81,0.63,-0.019,0.00,,C,B9
3,4,00 00 02.01,-51 53 36.8,8.06,7.75,0.97,0.370,0.43,,,F0V
4,5,00 00 02.39,-40 35 28.4,8.55,2.87,1.11,0.902,0.90,,,G8III


In [2]:
#displaying a summary of a data frame
hip_all_stars.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118218 entries, 0 to 118217
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Hip_No         118218 non-null  int64  
 1   Alpha          118218 non-null  object 
 2   Delta          118218 non-null  object 
 3   Vmag           118217 non-null  float64
 4   Plx            117955 non-null  float64
 5   e_Plx          117955 non-null  float64
 6   B-V            116937 non-null  float64
 7   V-I            116943 non-null  float64
 8   Var_period     118218 non-null  object 
 9   Var_type       118218 non-null  object 
 10  Spectral_type  118218 non-null  object 
dtypes: float64(5), int64(1), object(5)
memory usage: 9.9+ MB


###  Exploratory Data Analysis 
##### Stellar paralaxes 

Trigonometric stellar parallaxes in milliarcseconds are presented in the ['Plx'] column of the data frame. The parallax is an important stellar parameter because it is related to the stellar distance through a simple formula: [distance](in Parsecs) = 1/ [Plx] (in Arcseconds).

The ['Plx'] values are usually small, positive numbers. However, the parallaxes can appear as large numbers or as negative numbers and there are several reasons for that. One of the main reasons for the negative parallaxes is having errors ['e_Plx'] larger than the small values for the parallax or observing stars in the crowded regions of the sky. In any case, we will not consider stars with the negative parallaxes and therefore negative distances. Use the .copy() command when creating a new data frame to explicitly tell pandas that we are making a copy of a data frame. 

In [3]:
#Number of stars with no measured values for the parallax
no_plx = hip_all_stars['Plx'].isnull().sum()
print('Number of stars without measured parallaxes:{}'.format(no_plx) )

#Number of stars with negative parallaxes
negative_plx= sum([plx<0 for plx in hip_all_stars['Plx'].values])
print('Number of stars with negative parallaxes:{}'.format(negative_plx) )

#data frame with positive parallaxes
#to prevent the SettingWithCopyWarning due to hidden chaining use .copy()
hip_stars = hip_all_stars.loc[hip_all_stars['Plx']>0].copy()
hip_stars.shape

Number of stars without measured parallaxes:263
Number of stars with negative parallaxes:4196


(113710, 11)

##### Stellar magnitudes

In [4]:
#new column with the absolute magnitude

hip_stars['Mv'] = hip_stars['Vmag'] + 5 - 5*np.log10(1000/hip_stars['Plx'])

#rounding using data frame
hip_stars = hip_stars.round({'Mv': 2})

print(hip_stars.shape)
hip_stars.head(5)

(113710, 12)


,Hip_No,Alpha,Delta,Vmag,Plx,e_Plx,B-V,V-I,Var_period,Var_type,Spectral_type,Mv
0,1,00 00 00.22,+01 05 20.4,9.10,3.54,1.39,0.482,0.55,,,F5,1.85
1,2,00 00 00.91,-19 29 55.8,9.27,21.90,3.10,0.999,1.04,,C,K3V,5.97
2,3,00 00 01.20,+38 51 33.4,6.61,2.81,0.63,-0.019,0.00,,C,B9,-1.15
3,4,00 00 02.01,-51 53 36.8,8.06,7.75,0.97,0.370,0.43,,,F0V,2.51
4,5,00 00 02.39,-40 35 28.4,8.55,2.87,1.11,0.902,0.90,,,G8III,0.84


In [5]:
#Filtering stars brighter than the Sun

bright_df = hip_stars[hip_stars['Mv'] < 4.8]

print(bright_df.shape)

(104320, 12)


In [6]:
#Creating list of Hipparcos numbers for the stars brighter than the Sun

import timeit

%timeit -r2 -n10 Hip_list1 = [bright_df['Hip_No']]

%timeit -r2 -n10 Hip_list2 = list(bright_df['Hip_No'])

The slowest run took 16.84 times longer than the fastest. This could mean that an intermediate result is being cached.
33.8 µs ± 30 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
10.3 ms ± 732 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)


##### Spectral types for stars

The fastest way of counting how many stars belong to each spectral group is by using a special Python object called Counter. 

In [7]:
from collections import Counter

Sp_list = hip_all_stars['Spectral_type'].str.rstrip()

#create instance of the counter and examining run time
%timeit -r2 -n10 counter_dict = Counter(Sp_list)

#first three spectral groups
counter_dict = Counter(Sp_list)
print(list(counter_dict.items())[:3])

#printing the most common spectral groups 
print(counter_dict.most_common(3))
print(len(counter_dict))

17.1 ms ± 266 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
[('F5', 3861), ('K3V', 216), ('B9', 1581)]
[('K0', 8570), ('G5', 6017), ('A0', 4944)]
4125
